## This notebook is created to test the AI agent

from langchain_community.tools

### Define Tools

In [10]:
# search tools
from langchain_community.tools import DuckDuckGoSearchRun
from langchain.tools import BaseTool, StructuredTool, tool
from langchain_core.tools import tool
from langgraph.checkpoint.memory import MemorySaver

#search = DuckDuckGoSearchRun()
#search.invoke("How old is Brad Pitt?")

@tool
def search_tool(query: str) -> str:
    """Search the web for a query.
    
    Args:
        query: The query to search for.
    Returns:
        The result of the search.
    """
    search = DuckDuckGoSearchRun()
    return search.invoke(query)

@tool
def multiply_tool(a: int, b: int) -> int:
    """Multiply a and b.

    Args:
        a: first int
        b: second int
    Returns:
        The product of a and b.
    """
    return int(a) * int(b)

In [2]:
tools = [search_tool, multiply_tool]

In [3]:
tools

[StructuredTool(name='search_tool', description='Search the web for a query.\n\n    Args:\n        query: The query to search for.\n    Returns:\n        The result of the search.', args_schema=<class 'langchain_core.utils.pydantic.search_tool'>, func=<function search_tool at 0x000002B16C4F6950>),
 StructuredTool(name='multiply_tool', description='Multiply a and b.\n\n    Args:\n        a: first int\n        b: second int\n    Returns:\n        The product of a and b.', args_schema=<class 'langchain_core.utils.pydantic.multiply_tool'>, func=<function multiply_tool at 0x000002B16E5D5900>)]

### Define LLM API

In [4]:
import os

# Set Together.ai key as OPENAI_API_KEY, since Together uses OpenAI-compatible API
os.environ["OPENAI_API_KEY"] = "ab721cf1054a5db66582e22cc43aa90b069b7ee5c39d1f6cd4de48baef4ac06d"

from langchain.chat_models import init_chat_model

llm = init_chat_model(
    model="meta-llama/Llama-3.3-70B-Instruct-Turbo-Free",
    #model="deepseek-ai/DeepSeek-R1-Distill-Llama-70B-free",
    model_provider="together",  # important!
    temperature=0.1
)


In [5]:
llm_with_tools = llm.bind_tools(tools)

### Define Agent (graph) State

In [6]:
from typing import Annotated, Sequence, TypedDict
from langchain_core.messages import AnyMessage, BaseMessage, SystemMessage, HumanMessage
from langgraph.graph.message import add_messages

class AgentState(TypedDict):
    """Agent state for the graph.""" 
    messages: Annotated[Sequence[BaseMessage], add_messages]
    #history: Annotated[list[str], lambda x, y: x+y] # history trace

### Additonal features

In [7]:
from langchain_core.messages import AIMessage
import re

def extract_reasoning_chunks(text: str) -> list[str]:
    """Extract reasoning chunks (looks like) Thought, Action, Observation and Final Answer."""
    lines = text.splitlines()
    reasoning_chunks = [line.strip() for line in lines if re.match(r"^(Thought|Action|Observation|Final Answer):", line.strip())]
    return reasoning_chunks

#from langgraph.checkpoint.memory import MemorySaver
#memory = MemorySaver()

### Memory
#from langchain_core.chat_history import InMemoryChatMessageHistory
#from langchain_core.runnables.history import RunnableWithMessageHistory
#from uuid import uuid4

# Shared memory for the agent for session
# memory = ConversationBufferMemory(return_messages=True)
#chat_histories = {}

'''def get_session_history(session_id: str) -> InMemoryChatMessageHistory:
    if session_id not in chat_histories:
        chat_histories[session_id] = InMemoryChatMessageHistory()
    return chat_histories[session_id]'''

'def get_session_history(session_id: str) -> InMemoryChatMessageHistory:\n    if session_id not in chat_histories:\n        chat_histories[session_id] = InMemoryChatMessageHistory()\n    return chat_histories[session_id]'

### Define Agent (reasoner)

In [15]:
def agent(state: AgentState):
    """Agent invokes the llm with given query (human message) + system message
    and call tools if needed to get the response"""
    #MAX_MESSAGES = 1
    # parse the state
    messages = state["messages"] #[-MAX_MESSAGES:]

    system_message = SystemMessage(content=(
        "You are a helpful assistant who thinks step-by step before taking actions like using tools available to you.\n"
        "Use process: Thought → Action → Observation → Final Answer"
        "Only provide the final answer/response at the end without providing Thought, Action and Observation"
    ))

    # Get new LLm response
    llm_response = llm_with_tools.invoke([system_message] + messages)
    
    # Extract reasoning chunks from the response for history
    #reasoning_log = extract_reasoning_chunks(llm_response.content)
    
    response = {
        "messages": [llm_response]
        #"history": reasoning_log,
    }

    return response



In [9]:
### Custom condition function ###
def should_continue(state: AgentState) -> str:
    """Determine if we should use tools or end the conversation."""
    messages = state["messages"]
    last_message = messages[-1]
    
    # Check if the last message has tool calls
    if hasattr(last_message, "tool_calls") and last_message.tool_calls:
        return "tools"
    return "end"

### Buidling the agent graph

In [16]:
from langgraph.graph import START, StateGraph, END
from langgraph.prebuilt import tools_condition, ToolNode
from langchain_core.messages import ToolMessage

# Graph
workflow_graph = StateGraph(AgentState)

# Add nodes
workflow_graph.add_node("agent", agent)
workflow_graph.add_node("tools", ToolNode(tools))

# Add edges
workflow_graph.add_edge(START, "agent")
workflow_graph.add_conditional_edges("agent",
                                    #tools_condition # If tool calls are present, go to tools node 
                                    should_continue,
                                    {
                                        "tools": "tools",
                                        "end": END
                                    }
                                    )
workflow_graph.add_edge("tools", "agent")

# Run the graph
memory = MemorySaver()
react_agent = workflow_graph.compile(checkpointer=memory)


In [1]:
from IPython.display import Image, display
# Display the graph
display(Image(react_agent.get_graph(xray=True).draw_mermaid_png()))

NameError: name 'react_agent' is not defined

In [17]:
# response = react_agent.invoke({"messages": [HumanMessage(content="multiply the height of the Eiffel Tower by 5")]})

query = "Multiply the height of the Eiffel Tower by 10"
config3 = {"configurable": {"thread_id": "3"}}
response = react_agent.invoke({"messages": [HumanMessage(content=query)]}, config=config3)
response['messages'][-1].pretty_print()

================================== Ai Message ==================================

The Eiffel Tower's height is 324 meters. Multiplying this by 10 gives 3240. The answer is 3240.


In [18]:
response['messages'][-1].content

"The Eiffel Tower's height is 324 meters. Multiplying this by 10 gives 3240. The answer is 3240."

In [57]:
config6 = {"configurable": {"thread_id": "6"}}
#input_message1 = HumanMessage(content="What is the height of the Eiffel Tower ?")
input_message1 = HumanMessage(content="Hi My name is Jack !. Can you tell me the height of the Big Bell tower in London multiplied by 2 ?")
response1 = react_agent.invoke({"messages": [input_message1]}, config=config6)

In [58]:
response1['messages'][-1].pretty_print()

================================== Ai Message ==================================

Thought → The Big Bell tower in London is actually known as Big Ben, and its height is 96 meters. To find the height multiplied by 2, we need to perform the multiplication operation.
Action → We will use the multiplication tool to calculate the result of 96 * 2.
Observation → The result of the multiplication is 192.
Final Answer → The height of the Big Bell tower (Big Ben) in London multiplied by 2 is 192 meters.


In [63]:
response1['messages'][-1].content

'Thought → The Big Bell tower in London is actually known as Big Ben, and its height is 96 meters. To find the height multiplied by 2, we need to perform the multiplication operation.\nAction → We will use the multiplication tool to calculate the result of 96 * 2.\nObservation → The result of the multiplication is 192.\nFinal Answer → The height of the Big Bell tower (Big Ben) in London multiplied by 2 is 192 meters.'

In [59]:
for i in range(len(response1['messages'])):
    print(f"Response {i+1}:")
    response1['messages'][i].pretty_print() # Llama 3.3
    print("\n")

#response1['messages'][-1].pretty_print()

Response 1:
================================ Human Message =================================

Hi My name is Jack !. Can you tell me the height of the Big Bell tower in London multiplied by 2 ?


Response 2:
================================== Ai Message ==================================
Tool Calls:
  multiply_tool (call_5hfilmb7wo91aqelbxf25lwv)
 Call ID: call_5hfilmb7wo91aqelbxf25lwv
  Args:
    a: 96
    b: 2


Response 3:
================================= Tool Message =================================
Name: multiply_tool

192


Response 4:
================================== Ai Message ==================================

Thought → The Big Bell tower in London is actually known as Big Ben, and its height is 96 meters. To find the height multiplied by 2, we need to perform the multiplication operation.
Action → We will use the multiplication tool to calculate the result of 96 * 2.
Observation → The result of the multiplication is 192.
Final Answer → The height of the Big Bell tower (Bi

In [60]:
input_message2 = HumanMessage(content="What is my name ? and What is address of the Tower ?")
response2 = react_agent.invoke({"messages": [input_message2]}, config6)

In [61]:
for i in range(len(response2['messages'])):
    print(f"Response {i+1}:")
    response2['messages'][i].pretty_print() # Llama 3.3
    print("\n")
#response1['messages'][-1].pretty_print() # Llama 3.3

Response 1:
================================ Human Message =================================

Hi My name is Jack !. Can you tell me the height of the Big Bell tower in London multiplied by 2 ?


Response 2:
================================== Ai Message ==================================
Tool Calls:
  multiply_tool (call_5hfilmb7wo91aqelbxf25lwv)
 Call ID: call_5hfilmb7wo91aqelbxf25lwv
  Args:
    a: 96
    b: 2


Response 3:
================================= Tool Message =================================
Name: multiply_tool

192


Response 4:
================================== Ai Message ==================================

Thought → The Big Bell tower in London is actually known as Big Ben, and its height is 96 meters. To find the height multiplied by 2, we need to perform the multiplication operation.
Action → We will use the multiplication tool to calculate the result of 96 * 2.
Observation → The result of the multiplication is 192.
Final Answer → The height of the Big Bell tower (Bi

In [62]:
response2['messages'][-1].pretty_print() # Llama 3.3

================================== Ai Message ==================================

Thought → To answer your questions, I need to recall our previous conversation to find your name, and then search for the address of the Big Ben tower in London.
Action → I will recall our previous conversation to find your name, and then use the search tool to find the address of Big Ben.
Observation → Your name is Jack, and the address of Big Ben is Westminster, London SW1A 0AA.
Final Answer → Your name is Jack, and the address of the Big Ben tower in London is Westminster, London SW1A 0AA.


In [53]:
# Print full ReAct reasoning
print("\n--- Reasoning Trace ---")
for step in response["history"]:
    print(step)


--- Reasoning Trace ---


In [54]:
print("\n--- Reasoning Trace ---")
for step in response["messages"]:
    print(step)


--- Reasoning Trace ---
content='Multiply the height of the Eiffel Tower by 10' additional_kwargs={} response_metadata={} id='aa6b342c-f5f9-4812-baa1-679d1dd2d1d7'
content="<think>\nOkay, so I need to multiply the height of the Eiffel Tower by 10. Hmm, first, I should figure out how tall the Eiffel Tower actually is. I remember it's in Paris, and it's a really famous landmark. I think it's over 300 meters tall, but I'm not exactly sure of the exact number. Maybe I should look that up to get the precise measurement.\n\nWait, I think the Eiffel Tower is approximately 330 meters tall. Let me double-check that. Yeah, I've heard that number before, so I'll go with 330 meters. Now, the task is to multiply this height by 10. That sounds straightforward. So, 330 meters multiplied by 10. Let me do that calculation.\n\n330 times 10 is 3300. So, the result is 3300 meters. But wait, 3300 meters seems really tall. Let me make sure I did that right. 330 times 10 is indeed 3300. Yeah, that's correct

In [33]:
response['messages'][-1].pretty_print()

================================== Ai Message ==================================

<think>
First, I need to determine the current height of the Eiffel Tower. The Eiffel Tower stands at approximately 330 meters.

Next, I will multiply this height by 10 to find the result. 

So, 330 meters multiplied by 10 equals 3,300 meters.

Therefore, multiplying the height of the Eiffel Tower by 10 gives 3,300 meters.
</think>

**Solution:**

1. **Identify the height of the Eiffel Tower:**
   
   The Eiffel Tower is approximately **330 meters** tall.

2. **Multiply the height by 10:**
   
   \[
   330 \, \text{meters} \times 10 = 3,\!300 \, \text{meters}
   \]

3. **Final Answer:**
   
   \[
   \boxed{3300 \text{ meters}}
   \]


In [34]:
print(response['messages'][-1])

content='<think>\nFirst, I need to determine the current height of the Eiffel Tower. The Eiffel Tower stands at approximately 330 meters.\n\nNext, I will multiply this height by 10 to find the result. \n\nSo, 330 meters multiplied by 10 equals 3,300 meters.\n\nTherefore, multiplying the height of the Eiffel Tower by 10 gives 3,300 meters.\n</think>\n\n**Solution:**\n\n1. **Identify the height of the Eiffel Tower:**\n   \n   The Eiffel Tower is approximately **330 meters** tall.\n\n2. **Multiply the height by 10:**\n   \n   \\[\n   330 \\, \\text{meters} \\times 10 = 3,\\!300 \\, \\text{meters}\n   \\]\n\n3. **Final Answer:**\n   \n   \\[\n   \\boxed{3300 \\text{ meters}}\n   \\]' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 192, 'prompt_tokens': 44, 'total_tokens': 236, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'deepseek-ai/DeepSeek-R1-Distill-Llama-70B-free', 'system_fingerprint': None, 'id': 'npGLVk

In [15]:
len(response['messages'])

2

In [16]:
len(response['history'])

0

In [35]:
from langgraph.prebuilt import create_react_agent
graph = create_react_agent(llm, tools, checkpointer=MemorySaver())

In [36]:
config = {"configurable": {"thread_id": "thread-1"}}

def print_stream(graph, inputs, config):
     for s in graph.stream(inputs, config, stream_mode="values"):
         message = s["messages"][-1]
         if isinstance(message, tuple):
             print(message)
         else:
             message.pretty_print()

In [38]:
inputs = {"messages": [("user", "Hi I am Bob ?")]}
print_stream(graph, inputs, config)
print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
inputs2 = {"messages": [("user", "what is my name ?")]}
print_stream(graph, inputs2, config)

================================ Human Message =================================

Hi I am Bob ?
================================== Ai Message ==================================

<think>
Alright, the user just said "Hi I am Bob ?". That's a friendly greeting. I should respond in a warm and welcoming manner. Since they introduced themselves, I'll address them by name to make it personal. I should also offer assistance to show I'm here to help. Maybe something like, "Hi Bob! Welcome! How can I assist you today?" That should cover it and keep the conversation going smoothly.
</think>

Hi Bob! Welcome! How can I assist you today?
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
================================ Human Message =================================

what is my name ?
================================== Ai Message ==================================

<think>


You told me your name is Bob! 😊


In [2]:
%pip install --upgrade --quiet  langchain-community arxiv

Note: you may need to restart the kernel to use updated packages.


In [ ]:
from langgraph.prebuilt import create_react_agent
from langchain.agents import AgentExecutor, create_react_agent, load_tools
from langgraph.checkpoint.memory import MemorySaver
from langchain import hub
from langchain_core.messages import AnyMessage, BaseMessage, SystemMessage, HumanMessage

acadamic_tools = load_tools(
    ["arxiv"], search_tool
)

system_message = SystemMessage(content=(
        "You are a helpful assistant who thinks step-by step before taking actions like using tools available to you.\n"
        "Use process: Thought → Action → Observation → Final Answer"
        "Only provide the final answer/response at the end without providing Thought, Action and Observation"
    ))

#prompt = 

academic_agent = create_react_agent(llm, acadamic_tools, system_message)
agent_executor = AgentExecutor(agent=academic_agent, tools=acadamic_tools, verbose=True)



In [ ]:
agent_executor.invoke(
    {
        "input": "What's the paper NeuroSpex: Neuro-Guided Speaker Extraction with Cross-Modal Attention is about?",
    }
)

In [18]:
%pip install --quiet  docx2txt

Note: you may need to restart the kernel to use updated packages.


In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = PyPDFLoader("paper.pdf")
documents = loader.load()

splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
docs = splitter.split_documents(documents)

In [8]:
print(type(documents[0].page_content))

<class 'str'>


In [ ]:
docs

In [ ]:
from langchain_community.document_loaders import Docx2txtLoader

loader = Docx2txtLoader("fake.docx")
data = loader.load()

data

[Document(metadata={'source': 'fake.docx'}, page_content='Agenda Visit Haizhou Li\n\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\n\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\tMon, 21.4.2025\n\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t- 09:15 \t\tarrival Prof Li at the airport Bremen, Pick-up and transfer to hotel by car (Tanja)\n\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t- Prof. Li has various meetings during the day, so will stay in the hotel\n\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t- 17:00\t\t\tearly dinner in the hotel Munte restaurant, Margo reserved for 18:00 but we can walk in any time, they are open from 12 to 22:00 \n\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\n\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\tTue, 22.4.2025\n\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t- Pick-up at the hotel by car (Tanja), visit Oldenburg PIs and labs\n\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t- morning with Tania TBD\n\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t12:30\t\tMeet Prof. Radeloff at Haupteingang Uniklinik HNO,  call 0441 236-619 / Sekretariat -9692, he picks us up : Evangelisches

In [38]:
data[0].page_content =   clean_text(data[0].page_content)

In [42]:
dummy = ""

if dummy:
    print("dummy")
else:
    print("dummy is empty")

if data[0].page_content:
    print("text exsirs")
data[0].page_content

dummy is empty
text exsirs


'Agenda Visit Haizhou Li Mon, 21.4.2025  0915 arrival Prof Li at the airport Bremen, Pickup and transfer to hotel by car Tanja  Prof. Li has various meetings during the day, so will stay in the hotel  1700 early dinner in the hotel Munte restaurant, Margo reserved for 1800 but we can walk in any time, they are open from 12 to 2200 Tue, 22.4.2025  Pickup at the hotel by car Tanja, visit Oldenburg PIs and labs  morning with Tania TBD 1230 Meet Prof. Radeloff at Haupteingang Uniklinik HNO, call 0441 236619  Sekretariat 9692, he picks us up  Evangelisches Krankenhaus Oldenburg Steinweg 1317 1500 Meet Sandra Hellmers at the Uni Gebäude V04 1530 Tour OFFIS starting at 15301545 anbieten 1600 Meet Prof. Hein 1630 Meet Prof. Kollmeier at 1630 Uhr at NeSSy Wed, 23.4.2025 Full day at CSLMLL  meeting with Marvin  meeting with the MLL group, ethics committee meeting for the activeEars  lunch in the mensaunicum  further meeting with CSL members 1600  1745 Tanja BA Colloqium, IJCAI meeting 1800 dinne

In [40]:
import re

def clean_text(text: str) -> str:
    """
    Clean text by removing unwanted characters and normalizing the format
    args:
        text (str): raw text from document
    return:
        text (str): cleaned text
    """

    # Replace multiple spaces, tabs and newlines (\n) with a single space
    text = re.sub('\n', '', text)
    text = re.sub(r"\s+", " ", text)
    text = re.sub(' +', ' ', text)
    # Remove special characters except punctuation
    text = re.sub(r'[^\w\s.,!?]', '', text) 
    # remove leading and trailing spaces
    text = text.strip()

    return text